In [1]:
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
import sklearn
import data_process
import define_model

In [10]:
test = np.load("data/X_files_EPiC/X_test.npy")
label = np.load("data/X_files_EPiC/y_test.npy")

In [11]:
count = 0
for i in range(len(label)):
    if label[i]==1:
        count+=1
count

20000

In [12]:
print(np.shape(test))

(360000, 200, 4)
